In [2]:
import pandas as pd

df = pd.read_csv(f"tec2021_ext.csv")
df = df[["Avg. maximum unattested time", "Avg. worst case response time", "Avg. avg response time"]]
df

,Avg. maximum unattested time,Avg. time spent attesting,Avg. worst case response time,Avg. avg response time
0,425.0,852.0,350.00,66420.83
1,150.0,578.0,233.33,45116.67


In [4]:
from utils.serializer import deserialize_solution
from pathlib import Path
sol_noext = deserialize_solution(Path("testcases/output/RA_example_mode1_60_300_300_FFF_0.6/solution.pickle"))
sol_ext = deserialize_solution(Path("testcases/output/RA_example_mode2_60_300_1800_FFF_0.6/solution.pickle"))

In [12]:
from intervaltree import IntervalTree
from utils.utilities import sorted_complement

print("NOEXT")
total_time_attesting = 0
for es_prover in sol_noext.tc.ES_prover.values():
    ivs = []
    for t in sol_noext.tc.T_g[es_prover.id]:
        ivs.append((sol_noext.tc.schedule.o_t_val[t.id], sol_noext.tc.schedule.a_t_val[t.id]))
    iv_tree = IntervalTree.from_tuples(ivs)
    iv_tree.merge_overlaps(strict=False)
    iv_lengths = [iv.end - iv.begin for iv in iv_tree]
    slacks = sorted_complement(iv_tree, start=0, end=sol_noext.tc.hyperperiod)
    slack_lengths = [iv.end - iv.begin for iv in slacks]

    # Calculate the best block size for the maximum attestation time
    # Iterate through all slack sizes, choosing the one with maximum attestation time
    time_spent_attesting = 0
    block_length = 0
    slack_lengths_already_checked = set()
    for sl in slack_lengths:
        if sl not in slack_lengths_already_checked:
            cur_time_spent_attesting = 0
            for sl2 in slack_lengths:
                if sl <= sl2:
                    cur_time_spent_attesting += sl
            slack_lengths_already_checked.add(sl)
            if cur_time_spent_attesting > time_spent_attesting:
                time_spent_attesting = cur_time_spent_attesting
                block_length = sl

    print(f"{es_prover.id}")
    print(slacks)
    print(f"{(time_spent_attesting, block_length)}")
    total_time_attesting += time_spent_attesting
print(f"Total: {total_time_attesting}")
print()

print("EXT")
total_time_attesting = 0
for es_prover in sol_ext.tc.ES_prover.values():
    ivs = []
    for t in sol_ext.tc.T_g[es_prover.id]:
        ivs.append((sol_ext.tc.schedule.o_t_val[t.id], sol_ext.tc.schedule.a_t_val[t.id]))
    iv_tree = IntervalTree.from_tuples(ivs)
    iv_tree.merge_overlaps(strict=False)
    iv_lengths = [iv.end - iv.begin for iv in iv_tree]
    slacks = sorted_complement(iv_tree, start=0, end=sol_ext.tc.hyperperiod)
    slack_lengths = [iv.end - iv.begin for iv in slacks]

    # Calculate the best block size for the maximum attestation time
    # Iterate through all slack sizes, choosing the one with maximum attestation time
    time_spent_attesting = 0
    block_length = 0
    slack_lengths_already_checked = set()
    for sl in slack_lengths:
        if sl not in slack_lengths_already_checked:
            cur_time_spent_attesting = 0
            for sl2 in slack_lengths:
                if sl <= sl2:
                    cur_time_spent_attesting += sl
            slack_lengths_already_checked.add(sl)
            if cur_time_spent_attesting > time_spent_attesting:
                time_spent_attesting = cur_time_spent_attesting
                block_length = sl

    print(f"{es_prover.id}")
    print(slacks)
    print(f"{(time_spent_attesting, block_length)}")
    total_time_attesting += time_spent_attesting
print(f"Total: {total_time_attesting}")

NOEXT
Prover1
[Interval(0, 118), Interval(418, 2000)]
(1582, 1582)
Prover2
[Interval(0, 124), Interval(224, 800), Interval(1000, 2000)]
(1152, 576)
Prover3
[Interval(600, 2000)]
(1400, 1400)
Prover4
[Interval(600, 2000)]
(1400, 1400)
Total: 5534

EXT
Prover1
[Interval(0, 160), Interval(260, 528), Interval(628, 890), Interval(990, 2000)]
(1010, 1010)
Prover2
[Interval(0, 118), Interval(218, 466), Interval(566, 801), Interval(901, 2000)]
(1099, 1099)
Prover3
[Interval(200, 400), Interval(600, 800), Interval(1000, 2000)]
(1000, 1000)
Prover4
[Interval(200, 400), Interval(600, 800), Interval(1000, 2000)]
(1000, 1000)
Total: 4109
